In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

torch.cuda.device_count()
cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')
device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

### INIT

In [2]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Mono'
excl_folders = ['NoFX_mono', 'NoFX_mono_preprocessed']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=6

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
# set test_train_split=0.0 and val_train_split=0.0 to test pre-trained model
split = datasplit.DataSplit(dataset, test_train_split=0.8, val_train_split=0.1, shuffle=True)
# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

In [3]:
print('dataset size: ', len(dataset))
print('train set size: ', len(split.train_sampler))
print('val set size: ', len(split.val_sampler))
print('test set size: ', len(split.test_sampler))

dataset size:  163488
train set size:  117711
val set size:  13079
test set size:  32698


### TRAIN FxNET

In [4]:
# model
fxnet = models.FxNet(n_classes=dataset.num_fx).to(device)
# optimizer
optimizer_fxnet = optim.Adam(fxnet.parameters(), lr=0.001)
# loss function
loss_func_fxnet = nn.CrossEntropyLoss()

In [5]:
print(fxnet)

FxNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (batchNorm3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (batchNorm4): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=60, out_features=14, bias=True)
)


In [6]:
# SAVE
models_folder = 'saved_models'
model_name = '20201008_fxnet_best'
results_folder = 'saved_results'

In [7]:
# TRAIN and TEST FxNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

best_val_correct = 0

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainer.train_fx_net(
        model=fxnet,
        optimizer=optimizer_fxnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_fx_net(
        model=fxnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler, 
        device=device
    )
    
    test_loss, test_correct, test_results = trainer.test_fx_net(
        model=fxnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler, 
        device=device
    )
    # save models
    if val_correct > best_val_correct:
        best_val_correct = val_correct
        torch.save(fxnet, '%s/%s' % (models_folder, model_name))
        print('--- saved best model ---')
        
    # append results
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

Train Epoch: 0	[2000/117711 (2%)]	Total Loss: 49.9333	Avg Loss: 0.0250
Train Epoch: 0	[4000/117711 (3%)]	Total Loss: 88.9361	Avg Loss: 0.0222
Train Epoch: 0	[6000/117711 (5%)]	Total Loss: 122.3618	Avg Loss: 0.0204
Train Epoch: 0	[8000/117711 (7%)]	Total Loss: 152.3350	Avg Loss: 0.0190
Train Epoch: 0	[10000/117711 (8%)]	Total Loss: 180.2528	Avg Loss: 0.0180
Train Epoch: 0	[12000/117711 (10%)]	Total Loss: 204.9772	Avg Loss: 0.0171
Train Epoch: 0	[14000/117711 (12%)]	Total Loss: 228.4171	Avg Loss: 0.0163
Train Epoch: 0	[16000/117711 (14%)]	Total Loss: 249.1776	Avg Loss: 0.0156
Train Epoch: 0	[18000/117711 (15%)]	Total Loss: 269.8136	Avg Loss: 0.0150
Train Epoch: 0	[20000/117711 (17%)]	Total Loss: 289.2927	Avg Loss: 0.0145
Train Epoch: 0	[22000/117711 (19%)]	Total Loss: 307.5866	Avg Loss: 0.0140
Train Epoch: 0	[24000/117711 (20%)]	Total Loss: 325.8063	Avg Loss: 0.0136
Train Epoch: 0	[26000/117711 (22%)]	Total Loss: 343.0937	Avg Loss: 0.0132
Train Epoch: 0	[28000/117711 (24%)]	Total Loss: 3

KeyboardInterrupt: 

In [7]:
# BEST RESULTS
print('Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Accuracy:  91.10703332738656
Epoch:  47

Accuracy:  90.54209037388179
Epoch:  39

Accuracy:  90.38779130222032
Epoch:  42



In [14]:
# SAVE RESULTS
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
all_train_results_npy = np.array(all_train_results)

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
all_val_results_npy = np.array(all_val_results)

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
all_test_results_npy = np.array(all_test_results)

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_losses')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_correct')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_results')), arr=all_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_losses')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_correct')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_results')), arr=all_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_losses')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_correct')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_results')), arr=all_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'fx_labels')), arr=fx_labels_npy)